In [ ]:
# Goal: implement _very_ basic functionalities of pytorch from scratch (but I'm going to cheat and use numpy)

# large inspiration from:
# - https://github.com/karpathy/micrograd
# - https://github.com/geohot/tinygrad

In [2]:
import torch 
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms

In [54]:
# pytorch

# Create tensors.
x = torch.tensor(15., requires_grad=True)
print(x)
w = torch.tensor(10., requires_grad=True)
b = torch.tensor(900., requires_grad=True)

# Build a computational graph.
y = w * x + b * 100    # y = 2 * x + 3
print(y)

# Compute gradients.
# Computes the sum of gradients of given tensors with respect to graph leaves.
y.backward()

# Print out the gradients.
# Computes and returns the sum of gradients of outputs with respect to the inputs.
print(x.grad)    # x.grad = 2 = dy/dx = w
print(w.grad)    # w.grad = 1 = dy/dw = x
print(b.grad)    # b.grad = 1 = dy/db = 1

tensor(15., requires_grad=True)
tensor(90150., grad_fn=<AddBackward0>)
tensor(10.)
tensor(15.)
tensor(100.)


In [55]:
# not pytorch

class Tensor:
    def __init__(self, data, children=()):
        self.data = np.array(data, dtype=np.float32)
        self.children = children
        self.grad = 1
        self.op = None
        
    def __mul__(self, other):
        op = Multiply
        output = op.forward(self, other)
        output.op = op
        return output
    
    def __add__(self, other):
        op = Add
        output = op.forward(self, other)
        output.op = op
        return output    

    def backward(self):
        if self.op is not None:
            children_grads = self.op.backward(self, *self.children)
            for node, grad in zip(self.children, children_grads):
                node.grad = grad
        for node in self.children:
            node.backward()
            

class Multiply:
    def forward(a, b):
        return Tensor(np.multiply(a.data, b.data), (a, b))
    
    def backward(parent, a, b):
        return b.data * parent.grad, a.data * parent.grad
    
class Add:
    def forward(a, b):
        return Tensor(np.add(a.data, b.data), (a, b))
    
    def backward(parent, a, b):
        return parent.grad, parent.grad    

        

x = Tensor([15])
w = Tensor([10])
b = Tensor([900])
g = Tensor([100])
print(f'x: {x.data}')
print(f'w: {w.data}')
print(f'b: {b.data}')
print(f'')

y = w * x + b * g
print(f'y: {y.data}')

y.backward()

print(x.grad)
print(w.grad)
print(b.grad)

x: [15.]
w: [10.]
b: [900.]



AttributeError: 'int' object has no attribute 'data'

In [ ]:
        
#         self.op = Multiply
#         return self.op.forward(self, other)
    
#         out = Tensor(np.multiply(self.data, other.data), (self, other))
        
#         def _backward():
#             self.grad += other.data * out.grad
#             other.grad += self.data * out.grad
            
#         out._backword = _backward
        
#         return out
    
#     def __add__(self, other):
#         out = Tensor(np.add(self.data, other.data), (self, other))
        
#         def _backward():
#             self.grad += out.grad
#             other.grad += out.grad
        
#         return out
    
    
    
    
#         graph = []
        
#         def _build_graph(node):
#             for child in node._children:
#                 _build_graph(child)
#             graph.append(node)
        
#         _build_graph(self)
        
#         self.grad = 1